![book](https://raw.githubusercontent.com/ageron/tensorflow-safari-course/master/images/intro_to_tf_course.png)

**Try not to peek at the solutions when you go through the exercises. ;-)**

First let's make sure this notebook works well in both Python 2 and Python 3:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
tf.__version__

/home/farid/.virtualenvs/tensorflow-safari-course/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

*__From previous notebooks__*

In [3]:
learning_rate = 0.01
momentum = 0.8

# Using Readers

In [5]:
filenames = ["data/life_satisfaction.csv"]
n_epochs = 500

graph = tf.Graph()
with graph.as_default():
    reader = tf.TextLineReader(skip_header_lines=1)

    filename_queue = tf.train.string_input_producer(filenames, num_epochs=n_epochs)
    record_id, record = reader.read(filename_queue)

    record_defaults = [[''], [0.0], [0.0]]
    country, gdp_per_capita, life_satisfaction = tf.decode_csv(record, record_defaults=record_defaults)

In [6]:
batch_size = 5
with graph.as_default():
    X_batch, y_batch = tf.train.batch([gdp_per_capita, life_satisfaction], batch_size=batch_size)
    X_batch_reshaped = tf.reshape(X_batch, [-1, 1])
    y_batch_reshaped = tf.reshape(y_batch, [-1, 1])

In [7]:
with graph.as_default():
    X = tf.placeholder_with_default(X_batch_reshaped, shape=[None, 1], name="X")
    y = tf.placeholder_with_default(y_batch_reshaped, shape=[None, 1], name="y")

    b = tf.Variable(0.0, name="b")
    w = tf.Variable(tf.zeros([1, 1]), name="w")
    y_pred = tf.add(tf.matmul(X / 10000, w), b, name="y_pred")  # X @ w + b
    
    mse = tf.reduce_mean(tf.square(y_pred - y), name="mse")

    global_step = tf.Variable(0, trainable=False, name='global_step')
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(mse, global_step=global_step)

    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    saver = tf.train.Saver()

In [8]:
with tf.Session(graph=graph) as sess:
    init.run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    try:
        while not coord.should_stop():
            _, mse_val, global_step_val = sess.run([training_op, mse, global_step])
            if global_step_val % 100 == 0:
                print(global_step_val, mse_val)
    except tf.errors.OutOfRangeError:
        print("End of training")
    coord.request_stop()
    coord.join(threads)
    saver.save(sess, "./my_life_satisfaction_model")

100 0.310594
200 0.418606
300 0.0379472
400 0.0433003
500 0.142837
600 0.150865
700 0.113731
800 0.0544016
900 0.238013
1000 0.183196
1100 0.156038
1200 0.0214088
1300 0.245098
1400 0.346947
1500 0.28669
1600 0.0353458
1700 0.219129
1800 0.311164
1900 0.312836
2000 0.0508124
2100 0.199358
2200 0.353767
2300 0.314068
2400 0.119995
2500 0.186974
2600 0.352072
2700 0.331965
2800 0.141878
2900 0.0863107
End of training
